# Analyse exploratoire du jeu de données
## Fly me (chatbot)

<img src="./img/logo.png" style="text-align: center;">

Le but du projet est la création d'une MVP qui aidera les employés de Fly Me a réserver facilement un billet d'avion pour leurs vacances grâce a un chatbot qui aidrera les utilisateurs à choisir une offre de voyage.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os

In [2]:
df = pd.read_json("./data/frames.json")
print("Dimensionnalité : ", df.shape)
df.head()

Dimensionnalité :  (1369, 5)


,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


Voici les propriétés des colonnes : 

<table>
    <tr>
        <th>user_id</th>
        <th colspan="4">turns</th>
        <th>wizard_id</th>
        <th>id</th>
        <th colspan="2">labels</th>
    </tr>
    <tr style="text-align:center;">
        <td></td>
        <td>author</td>
        <td>text</td>
        <td>labels</td>
        <td>timestamp</td>
        <td></td>
        <td></td>
        <td>userSurveyRating</td>
        <td>wizardSurveyTaskSuccessful</td>
    </tr>
    <tr>
        <td>Désigne un identifiant unique pour l'utilisateur participant au dialogue.</td>
        <td>L'auteur du message dans un dialogue, c'est-à-dire "utilisateur" ou "assistant".</td>
        <td>La phrase que l'auteur a prononcé. C'est le texte exact que l'auteur d'un dialogue a dit. Par exemple, "texte" : "Considérez que c'est fait. Bon voyage !".</td>
        <td>Objet JSON qui possède trois clés : <br>  <br>  active_frame, acts, et acts_without_refs. <br> <br> La clé active_frame est l'identifiant de l'image actuellement active. Les actes sont les actes de dialogue pour l'énoncé actuel <br> <br> Chaque acte a un nom et des arguments args. <br> Le nom est le nom de l'acte de dialogue, par exemple, offrir, ou informer.  <br> Les arguments contiennent les types de slot (key) et les valeurs de slot (val), par exemple budget=2000$. Les valeurs des slots sont facultatives.  <br> <br> Un acte contient une balise ref chaque fois qu'un utilisateur ou un assistant fait référence à une image passée.<br><br> Les actes_without_refs sont similaires aux actes, sauf qu'ils n'ont pas ces balises ref. <br>  <br> Nous définissons la tâche de suivi de <br> trame comme la tâche qui prend en entrée les acts_without_refs et produit les actes.
        </td>
        <td>Horodatage Unix indiquant l'heure à laquelle le tour actuel s'est produit.</td>
        <td>Désigne un identifiant unique pour l'assistant qui participe au dialogue.</td>
        <td>Désigne une identification unique pour le dialogue.</td>
        <td>Une valeur qui représente la satisfaction de l'utilisateur à l'égard du service de l'assistant, allant de 1 - insatisfaction totale à 5 - satisfaction totale.</td>
        <td>Un booléen qui est vrai si l'assistant pense à la fin du dialogue que l'objectif de l'utilisateur a été atteint.</td>
    </tr>
</table>

source : https://www.microsoft.com/en-us/research/project/frames-dataset/download/

On peut ensuite analyser plus en détail la colonne labels puis turns.

In [3]:
df["labels"][0].keys()

dict_keys(['userSurveyRating', 'wizardSurveyTaskSuccessful'])

In [4]:
satisfaction_df = pd.json_normalize(df["labels"])
satisfaction_df.head(2)

,userSurveyRating,wizardSurveyTaskSuccessful
0,4.0,True
1,3.0,True


In [5]:
df["userSurveyRating"] = satisfaction_df["userSurveyRating"]
df["wizardSurveyTaskSuccessful"] = satisfaction_df["wizardSurveyTaskSuccessful"]

In [6]:
print("Il y a un total de", df[df["wizardSurveyTaskSuccessful"] == False].shape[0], "dialogues qui n'ont pas abouti à une réponse convenable de l'assistant.")

Il y a un total de 82 dialogues qui n'ont pas aboutie à une réponse convenable de l'assistant.


In [7]:
df.head(2)

,user_id,turns,wizard_id,id,labels,userSurveyRating,wizardSurveyTaskSuccessful
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc...",4.0,True
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc...",3.0,True


Ensuite même chose pour la colonne turns qui contient les informations sur les échanges bot/utilisateurs

In [8]:
turns_serie = df['turns']

In [9]:
turns_list = turns_serie.to_list()

In [10]:
turns_list[0][0].keys()

dict_keys(['text', 'labels', 'author', 'timestamp'])

In [11]:
turns_list[0][0]["text"]

"I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700."

In [12]:
turns_list[0][0]["labels"]

{'acts': [{'args': [{'val': 'book', 'key': 'intent'}], 'name': 'inform'},
  {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
    {'val': 'Caprica', 'key': 'or_city'},
    {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
    {'val': '8', 'key': 'n_adults'},
    {'val': '1700', 'key': 'budget'}],
   'name': 'inform'}],
 'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
   'name': 'inform'},
  {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
    {'val': 'Caprica', 'key': 'or_city'},
    {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
    {'val': '8', 'key': 'n_adults'},
    {'val': '1700', 'key': 'budget'}],
   'name': 'inform'}],
 'active_frame': 1,
 'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
    'budget': [{'val': '1700.0', 'negated': False}],
    'dst_city': [{'val': 'Atlantis', 'negated': False}],
    'or_city': [{'val': 'Caprica', 'negated': False}],
    'str_date': [{'val': 'august 13', 'negated': False}],
    'n_adults': 

In [13]:
turns_list[0][0]["labels"].keys()

dict_keys(['acts', 'acts_without_refs', 'active_frame', 'frames'])

In [14]:
turns_list[0][1]["text"]

'Hi...I checked a few options for you, and unfortunately, we do not currently have any trips that meet this criteria.  Would you like to book an alternate travel option?'

In [15]:
turns_list[1][0]["author"]

'user'

In [16]:
turns_list[0][0]["timestamp"]

1471272019730.0

In [17]:
dic_author = {}
for turns in turns_list:
    for message in turns:
        author = message["author"]
        if not author in dic_author.keys():
            dic_author[author] = 1
        else:
            dic_author[author] += 1

In [18]:
dic_author

{'user': 10407, 'wizard': 9579}

In [19]:
def get_key_value(arg):
    if "key" in arg and "val" in arg:
        if arg['val'] != "-1" and arg['val'] != None:
            key = arg["key"]
            val = arg["val"]
            return (key,val)
    return None

In [20]:
get_key_value(turns_list[0][0]["labels"]["acts"][1]["args"][2])

('str_date', 'Saturday, August 13, 2016')

In [21]:
all_value = {}
for conversation in turns_list[:1]:
    for message in conversation:
        # On récupére uniquement les echanges du user
        if message["author"] == "user":
            acts = message["labels"]["acts"]
            for act in acts:
                if act["args"]:
                    for arg in act["args"]:
                        key , value = get_key_value(arg)
                        if key in all_value:
                            all_value[key].append(value)
                        else:
                            all_value[key] = [value]

In [22]:
all_value

{'intent': ['book'],
 'dst_city': ['Atlantis', 'Neverland'],
 'or_city': ['Caprica', 'Atlantis'],
 'str_date': ['Saturday, August 13, 2016'],
 'n_adults': ['8', '5'],
 'budget': ['1700', '1900'],
 'ref': [[{'annotations': [{'val': 'Caprica', 'key': 'or_city'},
     {'val': 'August 13, 2016', 'key': 'str_date'}],
    'frame': 1,
    'fromrange': False}],
  [{'annotations': [{'val': 'Caprica', 'key': 'or_city'}],
    'frame': 1,
    'fromrange': False}]],
 'flex': [False]}

On remarque que les clés 'ref' ont une couche supplémentaire (pour rappel les 'refs' sont les références au message ultérieur).

Il convient donc d'appliquer des modifications pour mieux interpréter ce pattern

In [26]:
all_value = {}
for conversation in turns_list[:1000]:
    for message in conversation:
        # On récupére uniquement les echanges du user
        if message["author"] == "user":
            acts = message["labels"]["acts"]
            for act in acts:
                if act["args"]:
                    for arg in act["args"]:
                        if not arg["key"] == "ref":
                            data = get_key_value(arg)
                            if data:
                                key, value = data
                            if key in all_value:
                                all_value[key].append(value)
                            else:
                                all_value[key] = [value]
                        else:
                            # Les ref (on référence au conversation passé) sont à manipuler différement car il y a une couche supplémentaire
                            annotations_list = arg['val'][0]['annotations']
                            if annotations_list:
                                for annotation_value in annotations_list:
                                    data = get_key_value(annotation_value)
                                if data:
                                    key, value = data
                                if key in all_value:
                                    all_value[key].append(value)
                                else:
                                    all_value[key] = [value]

In [27]:
all_value.keys()

dict_keys(['intent', 'dst_city', 'or_city', 'str_date', 'n_adults', 'budget', 'flex', 'max_duration', 'n_children', 'end_date', 'seat', 'category', 'breakfast', 'seat_ok', 'gst_rating', 'downtown', 'spa', 'min_duration', 'wifi', 'duration', 'price', 'count', 'budget_ok', 'intent_ok', 'parking', 'name', 'beach', 'count_name', 'ref_anaphora', 'museum', 'park', 'impl_anaphora', 'mall', 'shopping', 'count_dst_city', 'cathedral', 'university', 'action'])

In [28]:
print("Clés et valeurs intéressantes pour notre contexte :")
print(" - ville d'arrivé (dst_city):",all_value["dst_city"][:5])

print(" - ville de départ (or_city):",all_value["or_city"][:5])

print(" - date du départ (str_date):",all_value["str_date"][:5])

print(" - date de retours (end_date):",all_value["end_date"][:5])

print(" - nombre de place adulte (n_adults):",all_value["n_adults"][:5])

print(" - Nombre de place enfant (n_children):",all_value["n_children"][:5])

print(" - Tarif du voyage maximum (budget):",all_value["budget"][:5])

print(" - classe des places dans l'avion (seat):",all_value["seat"][:5])

Clé et valeur intéressante pour notre contexte :
 - ville d'arrivé (dst_city): ['Atlantis', 'Neverland', 'Mos Eisley', 'Neverland', 'Mos Eisley']
 - ville de départ (or_city): ['Caprica', 'Atlantis', 'Gotham City', 'Gotham City', 'Birmingham']
 - date du départ (str_date): ['Saturday, August 13, 2016', 'Sunday August 21, 2016', '17th of August', 'Saturday, August 13, 2016', 'August 27']
 - date de retours (end_date): ['Wednesday August 31, 2016', 'Wednesday August 31, 2016', '31st', 'Tuesday, August 16, 2016', 'Wednesday, August 31, 2016']
 - nombre de place adulte (n_adults): ['8', '5', '5', '2', '2']
 - Nombre de place enfant (n_children): ['12', '6', 'four', 'four', 'four']
 - Tarif du voyage maximum (budget): ['1700', '1900', '2100', '$2500', '$2200']
 - classe des places dans l'avion (seat): ['economy', 'business', 'Business', 'upgrade', 'upgrade']


On peut retenir comme liste de clés utiles pour notre bot les clés suivantes : 
- 'dst_city'
- 'or_city'
- 'str_date'
- 'end_date'
- 'n_adult'
- 'n_children'
- 'budget'
- 'seat'

In [58]:
def format_data_luis(intent, text, labels):
    '''Cette fonction permet le formatage des dialogues au format de données LUIS
    
    Parameters:
        intent (str) : intention utilisateur, qui correspond au nommage LUIS pour l\'apprentissage, (définis le domaine d\'intention)
        text (str) : texte formulé par l\'utilisateur
        labels (list/tuple => tuple) : liste des labels récupérer du texte utilisateur avec la clé correspondante

    Returns:
        format_data (dict) : retourne un format de donnée utilisable par LUIS => 
        {
            \'texte\':\'le texte de l'utilisateur.\',
            \'intent_name\'=\'Exemple\',
            \'entity_labels\':[
                {
                    \'entity_name':\'wordText\',
                    \'start_char_index\':3,
                    \'end_char_index\':8
                }
            ]
        }
    '''
    text = text.lower()

    def entity_format(name, value):
        '''Cette fonction est une fonction interne pour formater les entity_label
        '''
        # on récupére la taille et le début de l'index de la value sur le text pour avoir le début et la fin de la value dans le texte
        value = value.lower()
        start = text.index(value)
        return dict(entity_name=name, start_char_index=start,
                    end_char_index=start + len(value))

    return dict(text=text, intent_name=intent,
                entity_labels=[entity_format(n, v) for (n, v) in labels])

In [59]:
def get_key_value(arg):
    '''On recupère la clé et valeur de l\'argument fourni pour les clés suivantes : "dst_city","or_city","str_date","end_date","n_adult","n_children","budget","seat"

    Parameters :
        arg (tuple) : argument de l\'intention utilisateur au format tuple.
        
    Returns:
        key_value (tuple) : retourne la clé/valeur de l\'argument au format tuple si il correspond à la liste de clé.
        OR
        None : si la valeur n'est pas formaté correctement ou que la clé n\'est pas dans la liste citée précédemment
    '''
    list_key = ['dst_city','or_city','str_date','end_date','n_adult','n_children','budget','seat']
    if "key" in arg and "val" in arg:
        if arg["key"] in list_key and arg['val'] != "-1" and arg['val'] != None:
            return (arg["key"],arg["val"])
    return None

In [51]:
final_data = []
for conversation in turns_list:
    for message in conversation:
        # On récupére uniquement les echanges du user
        if message["author"] == "user":
            text = message["text"]
            acts = message["labels"]["acts"]
            conversation_list = []
            for act in acts:
                if act["args"]:
                    for arg in act["args"]:
                        if not arg["key"] == "ref":
                            data = get_key_value(arg)
                            if data:
                                conversation_list.append(data)
                        else:
                            annotations_list = arg['val'][0]['annotations']
                            if annotations_list:
                                for annotation_value in annotations_list:
                                    data = get_key_value(annotation_value)
                                if data:
                                    conversation_list.append(data)
            if conversation_list:
                final_data.append(format_data_luis("BookFlight", text, conversation_list))


In [52]:
final_data[0]

{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
 'intent_name': 'BookFlight',
 'entity_labels': [{'entity_name': 'dst_city',
   'start_char_index': 27,
   'end_char_index': 35},
  {'entity_name': 'or_city', 'start_char_index': 41, 'end_char_index': 48},
  {'entity_name': 'str_date', 'start_char_index': 52, 'end_char_index': 77},
  {'entity_name': 'budget', 'start_char_index': 117, 'end_char_index': 121}]}